# Julia MNIST

In here I share how to run MNIST using Julia & flux.

In [97]:
import Pkg;Pkg.add("MLDatasets");Pkg.add("Flux");Pkg.add("Statistics");Pkg.add("ProgressMeter");Pkg.add("Plots");Pkg.add("Metal");Pkg.add("GZip");Pkg.add("MLUtils")

In [98]:
using MLDatasets, Flux, Statistics, ProgressMeter, Plots, Metal, MLUtils

In [99]:
using MLDatasets: MNIST
# load full training set
train_x, train_y = MNIST(split=:train, dir="mnist")[:]

# load full test set
test_x,  test_y  = MNIST(split=:test, dir="mnist")[:]

(features = Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; … ;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], targets = [7, 2, 1, 0, 4, 1, 4, 9, 5, 9  …  7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [100]:
train_y = Flux.onehotbatch(train_y, 0:9)
train_x = Float32.(train_x)
test_y = Flux.onehotbatch(test_y, 0:9)
test_x = Float32.(test_x)

train_loader = DataLoader((train_x, train_y), batchsize=32)
test_loader = DataLoader((test_x, test_y), batchsize=32)

313-element DataLoader(::Tuple{Array{Float32, 3}, OneHotArrays.OneHotMatrix{UInt32, Vector{UInt32}}}, batchsize=32)
  with first element:
  (28×28×32 Array{Float32, 3}, 10×32 OneHotMatrix(::Vector{UInt32}) with eltype Bool,)

In [101]:
# Flux.gpu_backend!("Metal")

In [102]:
model = Chain(
    Flux.flatten,
    Dense(784 => 128, relu),
    Dense(128 => 10),
    Flux.softmax
)# |> gpu

Chain(
  Flux.flatten,
  Dense(784 => 128, relu),              # 100_480 parameters
  Dense(128 => 10),                     # 1_290 parameters
  NNlib.softmax,
)                   # Total: 4 arrays, 101_770 parameters, 397.789 KiB.

In [103]:
optim = Flux.setup(Flux.Adam(0.01), model)  # will store optimiser momentum, etc.

(layers = ((), (weight = Leaf(Adam(0.01, (0.9, 0.999), 1.0e-8), (Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], (0.9, 0.999))), bias = Leaf(Adam(0.01, (0.9, 0.999), 1.0e-8), (Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], (0.9, 0.999))), σ = ()), (weight = Leaf(Adam(0.01, (0.9, 0.999), 1.0e-8), (Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], (0.9, 0.999))), bias = Leaf(Adam(0.01, (0.9, 0.999), 1.0e-8), (Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], (0.9, 0.999))), σ = ()), ()),)

In [104]:
for epoch in 1:10
    Flux.train!(model, train_loader, optim) do m, x, y
        y_hat = m(x)
        Flux.logitcrossentropy(y_hat, y)
    end
end

In [105]:
correct = 0
for (x, y) in test_loader
    y_hat = model(x)
    correct += sum(findmax(y_hat, dims=1)[2] .== findmax(y, dims=1)[2])
end

correct / size(test_y)[2]

0.9384